In [60]:
import pandas as pd
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LinearRegression
import re

# Read and clean homicide & green space data

In [61]:
url_hcount = "https://raw.githubusercontent.com/Amelie-Mad/green_spaces/refs/heads/main/Data/homicide_2018.csv"
homicide_2018 = pd.read_csv(url_hcount)
homicide_2018.loc[:, "COUNT"] = homicide_2018.COUNT.replace({":": None})
homicide_2018 = homicide_2018.dropna()
homicide_2018['COUNT'] = homicide_2018['COUNT'].apply(lambda x: int(x))

In [62]:
url_green = "https://raw.githubusercontent.com/Amelie-Mad/green_spaces/refs/heads/main/Data/FUA_location_map.csv"
green_2018 = pd.read_csv(url_green)
green_2018 = green_2018[['FUA code', 'FUA name', 'C_urban woody feature_FUA_%']]
green_2018 = green_2018.rename(columns={"FUA code": "FUA_CODE", "C_urban woody feature_FUA_%": "green"})
green_2018.green.isna().sum()

np.int64(0)

## Use official Eurostat linkage table to match FUA and NUTs3 regions

In [63]:
url_linkage_table = "https://raw.githubusercontent.com/Amelie-Mad/green_spaces/refs/heads/main/Data/ind_1629_fua_n3_data.csv"
# The data does not have headers. In fact, some of the columns are not very informative and merely contain a single observation
fua_nuts = pd.read_csv(url_linkage_table, header = None, names=['number','FUA_code','FUA_N3','NUTS','3','year','NUTS3', 'NUTS3_name','Description','link','FUA']) #read the official linkage table between FUA and NUTs3
fua_nuts.loc[:, 'FUA'] = fua_nuts.FUA.replace({"no FUA associated": None})
fua_nuts = fua_nuts.dropna()
fua_nuts['FUA_CODE'] = fua_nuts.FUA.apply(lambda x: re.sub(r'L\d+$', '', x))

In [64]:
df = green_2018.merge(fua_nuts[['NUTS3', 'FUA_CODE']], on = 'FUA_CODE')
df = df.merge(homicide_2018, left_on = "NUTS3", right_on = "CODE")
df.drop('CODE', axis = 1, inplace = True)
df['country'] = df['NUTS3'].apply(lambda x: x[0:2])
df

,FUA_CODE,FUA name,green,NUTS3,REGION,COUNT,country
0,AT001,Wien,27.006670,AT112,Nordburgenland,1,AT
1,AT001,Wien,27.006670,AT125,Weinviertel,1,AT
2,AT001,Wien,27.006670,AT126,Wiener Umland/Nordteil,2,AT
3,AT001,Wien,27.006670,AT127,Wiener Umland/Südteil,3,AT
4,AT001,Wien,27.006670,AT130,Wien,34,AT
...,...,...,...,...,...,...,...
543,SK004,Nitra,15.857479,SK023,Nitriansky kraj,5,SK
544,SK005,Prešov,49.872260,SK041,Prešovský kraj,2,SK
545,SK006,Žilina,65.374306,SK031,Žilinský kraj,7,SK
546,SK007,Trnava,22.495782,SK021,Trnavský kraj,16,SK


# Temperature data

In [65]:
# The countries for which we have temperature data
countries = ['Austria','Belgium','Denmark','France','Germany','Italy','Luxembourg','Netherlands','Poland','Portugal','Spain','Sweden','United_Kingdom']
# The columns we are interested in
columns = ['NUTS','Mean_AVG', 'Mean_Pre']
# The final dataframe with all the countries
meteo_df = pd.DataFrame(columns = columns)

The meteorological dataframes were too big to be uploaded on GitHub, so we store them locally and clean to select year 2018 exclusively, which is what we upload on github
```python
for country in countries:
    path = f"../Data (updated)/Meteorological indicator dataset for selected European NUTS 3 regions/{country}.csv"
    country_df = pd.read_csv(path, sep = ",")
    # Filter for year 2018
    country_df = country_df[country_df['YEAR'] == 2018]
    country_df.to_csv('../Data_final/Meteorological_datasets')
```

In [66]:
for country in countries:
    path = f"https://raw.githubusercontent.com/Amelie-Mad/green_spaces/refs/heads/main/Data/Meteorological_datasets/{country}.csv"
    country_df = pd.read_csv(path, sep = ",")
    # Filter for year 2018, already done in pre-cleaning before upload on GitHub
    # country_df = country_df[country_df['YEAR'] == 2018]
    # Filter for selected columns
    country_df = country_df[columns]
    # Yearly mean of temperature
    country_df = country_df.groupby('NUTS')[['Mean_AVG', 'Mean_Pre']].mean().reset_index()
    meteo_df = pd.concat([meteo_df, country_df])

/var/folders/t8/crqx384x59sbfjcs2flq1d380000gn/T/ipykernel_26408/585190642.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  meteo_df = pd.concat([meteo_df, country_df])


In [67]:
df = df.merge(meteo_df, left_on = 'NUTS3', right_on = 'NUTS')

# Population

## Absolute

In [68]:
df_pop = pd.read_excel('https://github.com/Amelie-Mad/green_spaces/raw/refs/heads/main/Data/population_original_data.xlsx', sheet_name = 2, header = 10)
df_pop = df_pop.iloc[:,:52]
drop_idx = list(range(3, df_pop.shape[1], 2))  # Starts from column 3 (index 2) and skips every second column
columns_to_drop = df_pop.columns[drop_idx]    # Get the corresponding column labels

# Drops the empty column after each year entry
df_pop = df_pop.drop(columns_to_drop, axis=1)
#Rename the columns
df_pop.columns = ['NUTS3', 'region'] + [str(x) for x in range (2000,2024)]
#Drop the NA rows
df_pop = df_pop[df_pop['2018'].isna() == False]
# Rename the columns with the population
df_pop.columns = list(df_pop.columns[:2]) + [f'{i}_pop' for i in df_pop.columns[2:]]

In [69]:
df = df.merge(df_pop[['2018_pop','NUTS3']], on = "NUTS3")

## Density

In [70]:
# Read the file, skip the empty lines at the beginning
df_pop_dens = pd.read_excel('https://github.com/Amelie-Mad/green_spaces/raw/refs/heads/main/Data/population_density_original_data.xlsx', sheet_name = 2, header = 8)
drop_idx = list(range(3, df_pop_dens.shape[1], 2))  # Starts from column 3 (index 2) and skips every other column
columns_to_drop = df_pop_dens.columns[drop_idx]    # Get the corresponding column labels
# Drops the empty column after each year entry
df_pop_dens = df_pop_dens.drop(columns_to_drop, axis=1)
# Rename the columns
df_pop_dens.columns = ['NUTS3', 'region'] + [str(x) for x in range (2000,2023)]
# Drop the NA rows
df_pop_dens = df_pop_dens[df_pop_dens['2018'].isna() == False] 
# Rename the columns with the population
df_pop_dens.columns = list(df_pop_dens.columns[:2]) + [f'{i}_pop_density' for i in df_pop_dens.columns[2:]]

/Users/amelie/Library/CloudStorage/OneDrive-epfl.ch/02_Statistics & Data science/Data science project/Codes-Reese/.venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [71]:
df = df.merge(df_pop_dens[['2018_pop_density','NUTS3']], on = "NUTS3")

# Education

In [72]:
# Load dataset
df_education = pd.read_excel('https://github.com/Amelie-Mad/green_spaces/raw/refs/heads/main/Data/education_original_data.xlsx', sheet_name = 2, header = 11)
# Starts from column 3 (index 2) and skips every other column
drop_idx = list(range(3, df_education.shape[1], 2))  
# Get the corresponding column labels
columns_to_drop = df_education.columns[drop_idx]    
# Drops the empty column after each year entry
df_education = df_education.drop(columns_to_drop, axis=1)
# Rename the columns
df_education.columns = ['NUTS2', 'region'] + [str(x) for x in range (2000,2024)]
# Drop the NA rows
df_education = df_education[df_education['2018'].isna() == False] 
# Rename the columns with the population
df_education.columns = list(df_education.columns[:2]) + [f'{i}_education_rate' for i in df_education.columns[2:]]

In [73]:
df['NUTS2'] = df['NUTS3'].str[:-1]
df = df.merge(df_education[['NUTS2','2018_education_rate']], on = 'NUTS2')

# GDP

In [74]:
# Load the GDP data
url = "https://github.com/Amelie-Mad/green_spaces/raw/refs/heads/main/Data/gdp_original_data.xlsx"
gdp_data = pd.read_excel(url, sheet_name=2, header=8)
drop_idx = list(range(3, gdp_data.shape[1], 2)) 
# Get the corresponding column labels
columns_to_drop = gdp_data.columns[drop_idx]    
# Drops the empty column after each year entry
gdp_data = gdp_data.drop(columns_to_drop, axis=1)
# Step 1: Drop any irrelevant or unnamed columns
gdp_data = gdp_data.apply(lambda x: x.replace({":": None}), axis = 1)

gdp_data.columns = ['NUTS3', 'region'] + [str(x) for x in range (2000,2023)]
gdp_data.columns = list(df_pop.columns[:2]) + [f'{i}_gdp' for i in gdp_data.columns[2:]]
gdp_data = gdp_data.dropna(subset=['2018_gdp']).reset_index(drop=True)

/Users/amelie/Library/CloudStorage/OneDrive-epfl.ch/02_Statistics & Data science/Data science project/Codes-Reese/.venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [75]:
df = df.merge(gdp_data[['NUTS3','2018_gdp']], on = "NUTS3")

In [76]:
df['area'] = df['2018_pop'] / df['2018_pop_density'] #in km squared 
df

,FUA_CODE,FUA name,green,NUTS3,REGION,COUNT,country,NUTS,Mean_AVG,Mean_Pre,2018_pop,2018_pop_density,NUTS2,2018_education_rate,2018_gdp,area
0,AT001,Wien,27.006670,AT112,Nordburgenland,1,AT,AT112,12.560150,1.859040,157840,96.9,AT11,28.4,5272.15,1628.895769
1,AT001,Wien,27.006670,AT125,Weinviertel,1,AT,AT125,11.735572,1.488058,124893,52.3,AT12,31.9,2957.82,2388.011472
2,AT001,Wien,27.006670,AT126,Wiener Umland/Nordteil,2,AT,AT126,12.167048,1.659725,327072,123.7,AT12,31.9,9474.42,2644.074373
3,AT001,Wien,27.006670,AT127,Wiener Umland/Südteil,3,AT,AT127,12.632928,1.821677,339977,236.3,AT12,31.9,17675.29,1438.751587
4,AT001,Wien,27.006670,AT130,Wien,34,AT,AT130,13.385606,1.724924,1888776,4780.6,AT13,42.3,97152.54,395.091829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,PT008,Aveiro,41.835130,PT16D,Região de Aveiro (NUTS 2021),3,PT,PT16D,15.249451,3.191520,364423,221,PT16,24,7164.63,1648.972851
402,PT009,Faro,33.569282,PT150,Algarve,6,PT,PT150,16.945434,1.352969,456113,89,PT15,22.2,9729.59,5124.865169
403,PT014,Viseu,59.653290,PT16G,Viseu Dão Lafões (NUTS 2021),1,PT,PT16G,14.744927,3.320604,254651,78.5,PT16,24,3789.93,3243.961783
404,PT016,Viana do Castelo,43.372124,PT111,Alto Minho,1,PT,PT111,14.608856,4.186990,232818,105.9,PT11,21.7,3600.31,2198.470255


# Aggregating on FUA

In [77]:
df_fua = df.groupby('FUA_CODE').agg(fua_name = ('FUA name', 'first'),
                                    country = ('country', 'first'),
                                    h_sum = ('COUNT', 'sum'), 
                                    green = ('green', 'mean'),
                                    mean_temp = ('Mean_AVG', 'mean'), 
                                    mean_pre = ('Mean_Pre', 'mean'), 
                                    sum_pop = ('2018_pop', 'sum'), 
                                    sum_area = ('area', 'sum'),
                                    sum_gdp = ('2018_gdp', 'sum'),
                                    mean_edu = ('2018_education_rate', 'mean')).reset_index()

When aggregating, we must pay attention to rate and ratio variables, which must not be averaged arithmetically. This is the case for crime rate, population density and GDP per capita. To avoid making this mistake, we keep all the components of these variables separate, add them up, and then calculate the variables of interest in a second stage.  That is, for a given city, we calculate:
-	Average crime rate by summing the crime counts occurring in relevant regions and dividing by the sum of the population of these regions. 
-	GDP per capita (in thousand euros) by summing GDP of the relevant regions and dividing by the sum of the population of these regions.
-	Average population density (in thousand inhabitants per $km^2$) by summing the population of relevant regions and dividing by the total area of the regions. 
The other variables (temperature, precipitation, education) are averaged arithmetically. Looking back, it could have been more accurate to do a weighted average based on area for precipitation and based on population size for the education level. 


In [78]:
df_fua['h_rate'] = (df_fua['h_sum'] / df_fua['sum_pop']) * 100000 # we convert crime per habitant into crime per hundred thousand inhabitants
df_fua['gdp_pp'] = df_fua['sum_gdp'] / df_fua['sum_pop'] * 1000 # GDP is in mio euros, so we arrange to make it in thousand euros per capita
df_fua['mean_pop_density'] = df_fua['sum_pop'] / (df_fua['sum_area'] * 1000) # population density in thousand persons per km^2

df_fua.drop(['sum_gdp', 'sum_pop', 'h_sum', 'sum_area'], axis = 1, inplace = True)


# Happiness

In [79]:
# Loading the data about green space satisfaction
green_satisfaction = pd.read_excel('https://github.com/Amelie-Mad/green_spaces/raw/refs/heads/main/Data/urb_percep__custom_13673917_page_spreadsheet.xlsx',
                                   sheet_name=2,
                                   header = 9)
# We drop every second column, which contains no information
drop_idx = list(range(2, green_satisfaction.shape[1], 2))
columns_to_drop = green_satisfaction.columns[drop_idx]
green_satisfaction = green_satisfaction.drop(columns_to_drop, axis=1)
# We rename the columns according to the year they correspond with
green_satisfaction.columns = ['city'] + [f"{year}_satisfaction" for year in ['2009', '2012', '2015', '2019', '2023']]

green_satisfaction.dropna(inplace = True)

# Replace : for missing values with NAs
green_satisfaction = green_satisfaction.apply(lambda x: x.replace({":": None}), axis = 1)

In [80]:
# We merge the data about green space satisfaction with the main dataframe. We manually adjust for different city names which allows us to merge on name, although this is suboptimal.
df_fua = df_fua.merge(green_satisfaction[['city','2019_satisfaction']], left_on = 'fua_name', right_on = 'city',how = 'left'
                    ).drop('city', axis = 1)

In [82]:
# We save the file locally before uploading it on github
df_fua.to_csv('full_sample.csv', index = False)